# Comparing three models for classification: zero-shot-classification, mistral and svm

In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import accuracy_score
import os
from openai import OpenAI
import openai

/home/joachim/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-17 15:58:47.917664: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-17 15:58:48.799360: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 15:58:48.799391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 15:58:48.889551: E 

In [9]:
ng = fetch_20newsgroups(shuffle=True,remove=('headers', 'footers', 'quotes'))

In [27]:
type(str(ng.data[0].split('\n')[:2]))

str

In [ ]:
for el in ng.data:
    ng = el.split('\n')[:2]

In [10]:
ng.target[5]

16

In [4]:
target_labels = ['atheism',
 'graphics',
 'ms-windows',
 'ibm.pc.hardware',
 'mac.hardware',
 'windows',
 'forsale',
 'autos',
 'motorcycles',
 'sport.baseball',
 'sport.hockey',
 'crypt',
 'electronics',
 'med',
 'space',
 'christian',
 'politics guns',
 'politics mideast',
 'politics misc',
 'religion misc']

In [8]:
# Clean data
import re

def clean_data(text):
    """This function cleans the input text by removing special characters and extra whitespace"""
    cleaned_text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)
    return cleaned_text

# Example usage
text = "From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If"

cleaned_text = clean_data(text)
print(cleaned_text)


From lerxstwamumdedu wheres my thing Subject WHAT car is this NntpPostingHost rac3wamumdedu Organization University of Maryland College Park Lines 15 I was wondering if anyone out there could enlighten me on this car I saw the other day It was a 2door sports car looked to be from the late 60s early 70s It was called a Bricklin The doors were really small In addition the front bumper was separate from the rest of the body This is all I know If


In [30]:
classifier = pipeline("zero-shot-classification")
pred = []
true = []
i = 0
for el in ng.data[:1000]:
  res = classifier(
        str(ng.data[i].split('\n')[:2]),
        candidate_labels=target_labels)
  true.append(ng.target[i])
  pred.append(target_labels.index(res['labels'][0]))
  i += 1

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


: 

In [14]:
accuracy_score(true, pred)

0.1

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)

/home/joachim/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 48.0/48.0 [00:00<00:00, 257kB/s]
config.json: 100%|██████████| 629/629 [00:00<00:00, 2.40MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 654kB/s]
model.safetensors: 100%|██████████| 268M/268M [05:46<00:00, 772kB/s] 


In [3]:
output

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [-3.6183,  3.9137]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [13]:
client = OpenAI(api_key="OPENAI_API_KEY")

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: OPENAI_A**_KEY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [9]:
model_name = "mistralai/Mixtral-8x7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )

tokenizer_config.json: 100%|██████████| 967/967 [00:00<00:00, 5.26MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 594kB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:02<00:00, 713kB/s]
special_tokens_map.json: 100%|██████████| 72.0/72.0 [00:00<00:00, 422kB/s]
config.json: 100%|██████████| 720/720 [00:00<00:00, 6.06MB/s]
model.safetensors.index.json: 100%|██████████| 92.7k/92.7k [00:00<00:00, 703kB/s]
model-00001-of-00019.safetensors:   1%|▏         | 62.9M/4.89G [01:41<2:09:41, 621kB/s]


KeyboardInterrupt: 

In [ ]:
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer = tokenizer, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

In [ ]:
prompt = "As a data scientist, can you explain the concept of regularization in machine learning?"

sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=100, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)
print(sequences[0]['generated_text'])